In [ ]:
import skimage.io as io
import numpy as np
import scipy
from skimage.morphology import binary_erosion, binary_dilation, binary_closing,skeletonize, thin
from skimage.filters import sobel_h, sobel, sobel_v, roberts, prewitt
from commonfunctions import *
from pip._internal import main as install
from pylab import imshow, gray, show 
import cv2
from math import pi
from scipy.ndimage import interpolation as inter
from statistics import mode,variance
from skimage.measure import find_contours
from skimage.filters import threshold_otsu
from skimage.util.shape import view_as_windows

In [ ]:
def showBinaryImg(img):
    cv2.imshow("Images",img*255)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()
def showImg(img):
    cv2.imshow("Images",img)
    cv2.waitKey(0) 
    cv2.destroyAllWindows()

In [ ]:
def customErosion(img,ST):
    countOnes= sum(sum(ST))
    imgConvolved = convolve2d(img,ST,mode="same")
    imgEroded = np.where(imgConvolved == countOnes,0,1)
    return imgEroded

def customDilation(img,ST):
    imgConvolved = convolve2d(img,ST,mode="same")
    imgDilated = np.where(imgConvolved >= 1,0,1)
    return imgDilated

In [45]:
def deskew(img,isSymbol=False,axis=1):
    if isSymbol:
        scoreArray = np.zeros(182)
        i = -45
        maxScore=0
        trueAngle=0
        while i <= 45:
            data = inter.rotate(img, i, reshape=False, order=0)
            sumRow = np.sum(data, axis=axis)
            score = np.sum((sumRow[1:] - sumRow[:-1]) ** 2)
            if(score>maxScore):
                maxScore=score
                trueAngle=i
            i+=1
        # print(trueAngle)
    else:
        scoreArray = np.zeros(362)
        i = 0
        while i <= 360:
            data = inter.rotate(img, i, reshape=False, order=0)
            sumRow = np.sum(data, axis=axis)
            score = np.sum((sumRow[1:] - sumRow[:-1]) ** 2)
            scoreArray[i] = score
            i+=1
        scoreArray = np.array(scoreArray)
        trueAngle = np.where(scoreArray == max(scoreArray))[0][0]
    rotated = inter.rotate(img, trueAngle, reshape=True, order=0)
    return rotated,trueAngle

def rotateBy(img,trueAngle):
    rotated = inter.rotate(img, trueAngle, reshape=True, order=0)
    return rotated


In [ ]:
#Binarization

def feng_threshold(img, w_size1=15, w_size2=30,
                   k1=0.15, k2=0.01, alpha1=0.1):
    """ Runs the Feng's thresholding algorithm.
    Reference:
    Algorithm proposed in: Meng-Ling Feng and Yap-Peng Tan, “Contrast adaptive
    thresholding of low quality document images”, IEICE Electron. Express,
    Vol. 1, No. 16, pp.501-506, (2004).
    Modifications: Using integral images to compute the local mean and the
    standard deviation
    @param img: The input image. Must be a gray scale image
    @type img: ndarray
    @param w_size1: The size of the primary local window to compute
        each pixel threshold. Should be an odd window
    @type w_size1: int
    @param w_size2: The size of the secondary local window to compute
        the dynamic range standard deviation. Should be an odd window
    @type w_size2: int
    @param k1: Parameter value that lies in the interval [0.15, 0.25].
    @type k1: float
    @param k2: Parameter value that lies in the interval [0.01, 0.05].
    @type k2: float
    @param alpha1: Parameter value that lies in the interval [0.15, 0.25].
    @type alpha1: float
    @return: The estimated local threshold for each pixel
    @rtype: ndarray
    """
    # Obtaining rows and cols
    rows, cols = img.shape
    i_rows, i_cols = rows + 1, cols + 1

    # Computing integral images
    # Leaving first row and column in zero for convenience
    integ = np.zeros((i_rows, i_cols), np.float)
    sqr_integral = np.zeros((i_rows, i_cols), np.float)

    integ[1:, 1:] = np.cumsum(np.cumsum(img.astype(np.float), axis=0), axis=1)
    sqr_img = np.square(img.astype(np.float))
    sqr_integral[1:, 1:] = np.cumsum(np.cumsum(sqr_img, axis=0), axis=1)

    # Defining grid
    x, y = np.meshgrid(np.arange(1, i_cols), np.arange(1, i_rows))

    # Obtaining local coordinates
    hw_size = w_size1 // 2
    x1 = (x - hw_size).clip(1, cols)
    x2 = (x + hw_size).clip(1, cols)
    y1 = (y - hw_size).clip(1, rows)
    y2 = (y + hw_size).clip(1, rows)

    # Obtaining local areas size
    l_size = (y2 - y1 + 1) * (x2 - x1 + 1)

    # Computing sums
    sums = (integ[y2, x2] - integ[y2, x1 - 1] -
            integ[y1 - 1, x2] + integ[y1 - 1, x1 - 1])
    sqr_sums = (sqr_integral[y2, x2] - sqr_integral[y2, x1 - 1] -
                sqr_integral[y1 - 1, x2] + sqr_integral[y1 - 1, x1 - 1])

    # Computing local means
    means = sums / l_size

    # Computing local standard deviation
    stds = np.sqrt(sqr_sums / l_size - np.square(means))

    # Obtaining windows
    padded_img = np.ones((rows + w_size1 - 1, cols + w_size1 - 1)) * np.nan
    padded_img[hw_size: -hw_size, hw_size: -hw_size] = img

    winds = view_as_windows(padded_img, (w_size1, w_size1))

    # Obtaining maximums and minimums
    mins = np.nanmin(winds, axis=(2, 3))

    # Obtaining local coordinates for std range calculations
    hw_size = w_size2 // 2
    x1 = (x - hw_size).clip(1, cols)
    x2 = (x + hw_size).clip(1, cols)
    y1 = (y - hw_size).clip(1, rows)
    y2 = (y + hw_size).clip(1, rows)

    # Obtaining local areas size
    l_size = (y2 - y1 + 2) * (x2 - x1 + 2)

    # Computing sums
    sums = (integ[y2, x2] - integ[y2, x1 - 1] -
            integ[y1 - 1, x2] + integ[y1 - 1, x1 - 1])
    sqr_sums = (sqr_integral[y2, x2] - sqr_integral[y2, x1 - 1] -
                sqr_integral[y1 - 1, x2] + sqr_integral[y1 - 1, x1 - 1])

    # Computing local means2
    means2 = sums / l_size

    # Computing standard deviation range
    std_ranges = np.sqrt(sqr_sums / l_size - np.square(means2))

    # Computing normalized standard deviations and extra alpha parameters
    n_stds = stds / std_ranges
    n_sqr_std = np.square(n_stds)
    alpha2 = k1 * n_sqr_std
    alpha3 = k2 * n_sqr_std

    thresholds = ((1 - alpha1) * means + alpha2 * n_stds
                  * (means - mins) + alpha3 * mins)
    for x in range(img.shape[0]):
        for y in range(img.shape[1]):
            if(img[x][y]<thresholds[x][y]):
                img[x][y]=0
            else:
                img[x][y]=1
    return img


In [ ]:
#Smoothing
def signaltonoise(a, axis=None, ddof=0):
    a = np.asanyarray(a)
    m = a.mean(axis)
    sd = a.std(axis=axis, ddof=0)
    return np.where(sd == 0, 0, m/sd)
def smooth(gray_img):   
    snr = signaltonoise(gray_img)
    if(snr<1):
        snr=1
    if(snr>5):
        snr=5
    sigma=(-50/4)*(snr-1)+60
    smImage=cv2.bilateralFilter(gray_img,8,int(sigma),int(sigma))
    return smImage

In [ ]:
#Illumenation
def simplestColorBalance(img,s):
    s/=100
    n,m=img.shape
    f=img.flatten()
    f.sort()
    minT=f[int(s*n*m)]
    maxT=f[ n*m - int(s*n*m) - 1]
    img=np.where(img<minT,0,img)
    img=np.where(img>maxT,255,img)
    img=np.where((img!=0)&(img!=255),255*((img-minT)/(maxT-minT)),img)
    return img
def poisonScreening(img,L):
    n,m=img.shape
    img_freq=cv2.dft(img)
    for i in range(n):
        for j in range(m):
            coef=(pi*pi*i*i)/(n*n) + (pi*pi*j*j)/(m*m)
            img_freq[i][j]=(img_freq[i][j]*coef)/(L+coef)
    img=cv2.idft(img_freq)
    return img
def applyPoison(img,s,L):
    img=simplestColorBalance(img,s)
    cv2.imwrite('Balanced.png',img)
    img=poisonScreening(img,L)
    cv2.imwrite('poison.png',img)
    img=simplestColorBalance(img,s)
    cv2.imwrite('final.png',img)
    return img


In [ ]:
#determine if the image needs illumination evening and if it need feng thresholding 
#if image has uneven illumination => use poisson
#if image has uneven illumination or a very low contrast(another function not done yet) => use feng
#Return True if image is good and false if image is uneven
def imageState(imgGray):
    blurred = cv2.GaussianBlur(imgGray, (25, 25), 0)
    no_text = imgGray * ((imgGray/blurred)>0.99)                
    no_text[no_text<10] = no_text[no_text>20].mean()      
    no_bright = no_text.copy()
    no_bright[no_bright>220] = no_bright[no_bright<220].mean()
    std = no_bright.std()
    bright = (no_text>220).sum()
    if std>18 or (no_text.mean()<200 and bright>8000):
        return False
    else:
        return True


In [ ]:
def calcStaffHeightSpace(img):
    count=0
    staffHeight=[]
    staffSpace=[]
    n,m=img.shape
    lines=[]
    for j in range(m):
        start=end=0
        for i in range(n-1):
            if(img[i][j]!=img[i+1][j]):
                if img[i][j]==0:
                    #end of zeros
                    start=i
                    staffSpace.append(count)
                else:
                    #end of ones
                    end=i
                    lines.append([j,start,end,count])#colIndex,start in coloum Index,end in coloum index,thikness
                    staffHeight.append(count)
                count=0
            else:
                count+=1
        
                       
    staffHeight=mode(staffHeight)
    staffSpace=mode(staffSpace)
    return lines,staffSpace,staffHeight

In [47]:
def removeSymbol(img,lines,staffHeight,staffSpace):
    Thres=min(staffHeight+staffSpace,2*staffHeight)
    lines=np.array(lines)
    indices=np.where(lines[:,3]>Thres)
    staffImg=np.copy(img)
    for i in indices[0]:
        col,start,end,thres=lines[i]
        staffImg[start:end+1,col]=0
    return staffImg

def removeStaffInitial(img):
    imgLines,staffS,staffH=calcStaffHeightSpace(img)
    imgRemovedSymbols=removeSymbol(img,imgLines,staffH,staffS)
    neg=img-imgRemovedSymbols
    neg=neg.astype(np.float)
    # ST=np.ones((5,5))
    # neg=cv2.dilate(neg,ST)
    neg =1-neg
    cv2.imwrite("neg.png",neg*255)
    return staffS,staffH,neg

In [ ]:
#divide binarized image into segments
#get Horizontal Projection 
# imgCopy= np.copy(BinarizedImage)
# img=imgCopy
# img=1-img
def rowProjection(img):
    proj = np.sum(img,1)
    maxProjection = np.max(proj) 
    result = np.zeros(img.shape) 
    # Draw a line for each row 
    for row in range(img.shape[0]):
        result[row,0:int(proj[row])]=1  
    result=1-result
    cv2.imwrite("result.png",result*255)
    return maxProjection,result

In [ ]:
#estimate staff position 
def calcStaffPos(rowHist,maxProjection,thr):
    n,m=rowHist.shape
    thres=int(maxProjection*thr)
    start=end=0
    width=[]
    peaksMid=[]
    for i in range(n-1):
        if(rowHist[i][thres]!=rowHist[i+1][thres]):
            if rowHist[i][thres]==1:
                #end of zeros
                start=i
            else:
                #end of ones
                end=i+1
                peaksMid.append(int((start+end)/2))
                width.append((end-start))
    return np.array(width),np.array(peaksMid)

#estimate staff segment position 
#TODO: try to make segments into blocks with ST
def calcSegmentPos(rowHist,thres):
    n,m=rowHist.shape
    start=end=0
    width=[]
    peaksMid=[]
    for i in range(n-1):
        if(rowHist[i][thres]!=rowHist[i+1][thres]):
            if rowHist[i][thres]==1:
                #end of zeros
                start=i
            else:
                #end of ones
                end=i+1
                peaksMid.append(int((start+end)/2))
                width.append((end-start))
    return np.array(width),np.array(peaksMid)

def filterPeaks(peaksMids,widths):
    if(len(peaksMids)<=1):
        return widths,peaksMids
    newPeaks=[]
    newWidth=[]
    sumW=0
    for i in widths:
        sumW+=i
    avgW=sumW/len(widths)
    #if i dont have flase peaks for clusters then this will result in a problem
    var = variance(widths)
    for i in range(len(peaksMids)):
        if(widths[i]>=avgW or (var<1000 and (avgW-widths[i])<20)):
            newPeaks.append(peaksMids[i])
            newWidth.append(widths[i])
    return newWidth,newPeaks

#divide image into segments
def imgStaffSegments(img,segMids,widths,staffS,originalImg,returnOringal):
    segments=[]
    segmentsOriginal=[]
    staffS=int(staffS)
    n,m=img.shape
    for i in range(len(segMids)):
        up=0
        down=n
        if(segMids[i]-int(widths[i]/2)-2*staffS > 0):
            up =segMids[i]- int(widths[i]/2) - 2*staffS
        if(segMids[i]+int(widths[i]/2)+2*staffS<n):
            down=segMids[i]+int(widths[i]/2)+2*staffS
        sliced= img[up:down,:]
        if(sliced.shape[0] != 0 and sliced.shape[1] != 0):
            segments.append(sliced)
            if(returnOringal):
                slicedOriginal= originalImg[up:down,:]
                segmentsOriginal.append(slicedOriginal)

    if(returnOringal):
        return segments,segmentsOriginal
    else:
        return segments

def maxStaffSpace(peaksMids,width):
    maxSpace=0
    for i in range(len(peaksMids)):
        if(i%5<4 and i+1<len(peaksMids)and peaksMids[i+1]-peaksMids[i]>maxSpace):
            maxSpace=peaksMids[i+1]-peaksMids[i]
            maxSpace=maxSpace-(width[i]/2+width[i+1]/2)
    return maxSpace



In [ ]:
#loop in array of images and remove staff from each
def removeStaffRow(imgOriginal,midPoint,curWidth):
    thresPixel=curWidth
    for i in range(imgOriginal.shape[1]):
        pixelSum= sum(imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1])
        if(pixelSum<=thresPixel):
            imgOriginal[midPoint-curWidth:midPoint+curWidth,i:i+1]=0
    return imgOriginal

In [ ]:
#discard smaller contours overlapped with another bigger contours
#reorder contours from left to right and from top to dowm
def checkOverlapped(c,imgContours):
    cXmin,cXmax,cYmin,cYmax = c
    for Xmin,Xmax,Ymin,Ymax in imgContours:
        if(cXmin > Xmin and cXmax < Xmax and cYmin > Ymin and cYmax < Ymax):
            return True
    return False
def filterContours(imgContours):
    filtered=[]
    for c in imgContours:
        if(not checkOverlapped(c,imgContours)):
            filtered.append(c)
    filtered.sort()
    return filtered

def isNum(imgContours):
    isNUMList = np.zeros(len(imgContours))
    for index in range(len(imgContours)-1):
        XminC,XmaxC,YminC,YmaxC = imgContours[index]
        XminN,XmaxN,YminN,YmaxN = imgContours[index+1]
        width = XmaxC - XminC
        height = YmaxC - YminC
        ##### first condition xminC , xminN , xmaxC, xmaxN are nearly equal#####
        #####second condition ymaxC < yminN #########
        isnotNUM = height / width > 4
        if(abs(XmaxC - XminC) >= 3 and abs(XmaxN - XminN) >= 3 and YmaxC < YminN and not isnotNUM):
            isNUMList[index] = 1
            isNUMList[index + 1] = 1
    return isNUMList

In [43]:
#convert image into contours
counter=0
def getContoursDeskewed(imgWithStaff,imgWithoutStaff):
    contours = find_contours(imgWithoutStaff, 0.8)
    imgContours=[]
    imgSymbols=[]
    imgSymbolsNoStaff=[]
    index=0
    for contour in contours:
        x = contour[:,1]
        y = contour[:,0]
        [Xmin, Xmax, Ymin, Ymax] = [np.amin(x), np.amax(x), np.amin(y), np.amax(y)]
        imgContours.append([Xmin, Xmax, Ymin, Ymax])

    h,w = imgWithStaff.shape
    imgContours = filterContours(imgContours)
    isNUMList = isNum(imgContours)
    for Xmin,Xmax,Ymin,Ymax in imgContours:
        imgSymbol = imgWithStaff[0:h,int(Xmin):int(Xmax+1)]
        imgSymbolNoStaff = imgWithoutStaff[0:h,int(Xmin):int(Xmax+1)]

        imgSymbol = 1 -imgSymbol
        thinned = thin(imgSymbol)
        thinned,angle = deskew(thinned,True,1)
        imgSymbol = rotateBy(imgSymbol,angle)
        imgSymbol = 1 - imgSymbol

        #get vertical images

        imgSymbolNoStaff = 1 -imgSymbolNoStaff
        thinned = thin(imgSymbolNoStaff)
        thinned,angle = deskew(thinned,True,0)
        imgSymbolNoStaff = rotateBy(imgSymbolNoStaff,angle)
        imgSymbolNoStaff = 1 -imgSymbolNoStaff

        imgSymbols.append(imgSymbol)
        imgSymbolsNoStaff.append(imgSymbolNoStaff)
        index+=1
    return imgSymbols,imgSymbolsNoStaff,imgContours,isNUMList


In [35]:
def staffRemovalNonHorizontal(BinarizedImage):
    staffS,staffH,staffFree = removeStaffInitial(1-BinarizedImage)
    maxProjection,result=rowProjection(1-staffFree)
    #estimate staff segment position
    segWidth,segMids=calcSegmentPos(result,60)
    #filter peaks
    segWidth,segMids=filterPeaks(segMids,segWidth)
    #divide image into segments and divide image => return array of images each image has staff segment of staff removed and original one
    imgSegments,imgSegmentsStaff = imgStaffSegments(staffFree,segMids,segWidth,staffS,BinarizedImage,True)
    segContours=[]
    segContoursDim=[]
    checkNumList=[]
    segContoursPeakmids=[]
    segContoursWidth=[]
    i=0
    j=0
    for img in imgSegmentsStaff:
        cv2.imwrite("segments/"+str(i)+"_"+str(j)+".png",img*255)
        j+=1

    for imgSeg in imgSegments:
        imgContours,imgContoursNoStaff,imgContoursDim,isNum = getContoursDeskewed(imgSegmentsStaff[i],imgSeg)
        ###############################################for testing###################################################
        j=0
        for img in imgContoursNoStaff:
            cv2.imwrite("staffRemoved/"+str(i)+"_"+str(j)+".png",img*255)
            j+=1
        j=0
        for img in imgContours:
            cv2.imwrite("contours/"+str(i)+"_"+str(j)+".png",img*255)
            j+=1
        ###################################################
        segContourWidth=[]
        segContourPeakmids=[]
        #remove staff from array of imgs
        for cimg in imgContours:
            cimg=1-cimg
            maxProjection,result=rowProjection(cimg)
            staffWidth,peaksMids=calcStaffPos(result,maxProjection,0.8)
            segContourWidth.append(staffWidth)
            segContourPeakmids.append(peaksMids)
        segContoursDim.append(imgContoursDim)
        segContours.append(imgContoursNoStaff)
        checkNumList.append(isNum)
        segContoursPeakmids.append(segContourPeakmids)
        segContoursWidth.append(segContourWidth)
        i+=1
    return segContours,segContoursDim,staffS,checkNumList,segContoursPeakmids,segContoursWidth

In [ ]:
img= cv2.imread("18.jpg")
countourImg=np.copy(img)
gray_img=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
isEven = imageState(gray_img)
IlluminatedImage=gray_img
if(not isEven):
    IlluminatedImage=applyPoison(gray_img,0.1,0.1)
    IlluminatedImage=IlluminatedImage.astype(np.uint8)
smoothedImage=smooth(IlluminatedImage)
if  not isEven:
    BinarizedImage=feng_threshold(smoothedImage)
else:
    t =threshold_otsu(smoothedImage)
    BinarizedImage=np.where(smoothedImage>t,1,0)
BinarizedImage=1-BinarizedImage
RotatedImage,angle=deskew(BinarizedImage)
BinarizedImage=1-RotatedImage

In [48]:
staffRemovalNonHorizontal(BinarizedImage)

.2, 421.0],
   [571.2, 598.8, 327.2, 363.8],
   [574.2, 679.8, 110.2, 355.8],
   [723.2, 726.8, 265.2, 281.8],
   [733.2, 760.8, 235.2, 269.8],
   [742.2, 748.8, 190.2, 207.8],
   [742.2, 768.8, 248.2, 300.8],
   [751.2, 752.8, 186.2, 196.8],
   [753.2, 778.8, 161.2, 196.8],
   [761.2, 786.8, 176.2, 225.8],
   [837.2, 919.8, 189.2, 347.8],
   [1226.2, 1338.8, 147.2, 330.8],
   [1328.2, 1383.8, 154.2, 312.8],
   [1375.2, 1392.8, 177.2, 197.8],
   [1384.2, 1385.8, 154.2, 164.8],
   [1395.2, 1404.8, 154.2, 183.8],
   [2019.2, 2120.8, 178.2, 337.8],
   [2223.2, 2326.8, 177.2, 337.8],
   [2329.2, 2433.8, 149.2, 336.8],
   [2459.2, 2471.8, 155.2, 316.8],
   [2583.2, 2646.8, 130.2, 333.8],
   [2802.2, 2861.8, 160.2, 327.8],
   [3027.2, 3241.8, 149.2, 351.8],
   [3336.2, 3446.8, 58.2, 257.8],
   [3461.2, 3571.8, 159.2, 335.8],
   [3581.2, 3691.8, 80.2, 285.8],
   [3855.2, 3880.8, 71.2, 249.8],
   [4035.2, 4056.8, 401.2, 421.0],
   [4069.2, 4104.8, 58.2, 240.8],
   [4089.2, 4144.8, 57.2, 240.8]